In [5]:
import requests
import urllib.parse
import json
from time import time
import pandas
import collections
import math
import collections
from googletrans import Translator
import re
import nltk
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import time

import requests
import urllib.parse

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation


class RefEnv(object):
    """docstring for RefEnv"""
    def __init__(self):
        self.dst = 'Source'
        self.Export_folder = 'Export'
        self.src = '/Users/py3/Google Drive (mohamed.diabakhate@digital-africa.co)/PROJET DATA/DATA/{}'
        self.transformed = '/Users/py3/Google Drive (mohamed.diabakhate@digital-africa.co)/PROJET DATA/DATA/Transformed/{}'
        

class Transformation(object):
    """docstring for RefEnv"""
    def __init__(self):
        self.src = RefEnv().src.format('Transformed/df1_clean.csv')
        self.X = pandas.read_csv(self.src).fillna('VIDE')
        #self.export = X.to_csv(RefEnv().src.format('Transformed/prep_df1_clean.csv'))
        self.header = ['cat_struc', 'cat_autre_struc']
        self.header_tr = ['prez_struc', 'prez_produit_struc', 'prez_marche_struc', 'prez_zone_struc', 'prez_objectif_struc', 'prez_innovante_struc', 'prez_duplicable_struc', 'prez_durable_struc'] 

    def categorie_structure(self,cat_struc,cat_autre_struc):
        if cat_struc == "Santé, bien être, et protection sociale" or cat_struc == "Health, well-being and social protection":
            return "sante" 
        if cat_struc == "Fintech":
            return "finance"
        if cat_struc == "Technologies émergentes" or cat_struc == "Emerging technologies":
            return "technologie"
        if cat_struc == "Médias, édition et communication" or cat_struc =="Media, publishing and communication":
            return "media"
        if cat_struc == "Agriculture, territoires ruraux et ressources naturelles" or cat_struc == "Agriculture, rural areas and natural resources":
            return  "agriculture"
        if cat_struc == "Education, formation professionnelle et emploi" or cat_struc =="Education, vocational training and employment":
            return  "education" 
        if cat_struc == "E-commerce, marketing et distribution" or cat_struc == "E-commerce, marketing and distribution":
            return  "ecommerce"
        if cat_struc == "Gouvernement ouvert et transformation sociale" or cat_struc == "Open government and social transformation":
            return  "gouvernement"
        if cat_struc == "Energie" or cat_struc == "Energy":
            return  "energie" 
        if cat_struc == "Territoires intelligents et mobilité" or cat_struc == "Intelligent territories and mobility":
            return  "mobilite"
        if cat_struc == "Industries culturelles et créatives" or cat_struc == "Cultural and creative industries":
            return  "culture"
        if (cat_struc == "Autre (précisez)" or cat_struc == "Other - please specify" or cat_struc == "autre (précisez)"):
            return  'autres'
        if cat_struc != cat_struc:
            return 'non renseigné'
        else:
            return cat_struc
    
    def traductor(self,element):
        key = '9b87800e-c11c-5e98-649b-21d40847e9af'
        text= urllib.parse.quote(element)
        url = 'https://api.deepl.com/v2/translate?auth_key={}&text={}&target_lang=EN'.format(key, text)
        return url

    def lower_case(self, element):
        return element.lower()

    def number_to_text(self, element):
        return re.sub(r'\d+','', element)

    def remove_punctuation(self, element):
        return element.translate(str.maketrans('', '', string.punctuation))

    def remove_whitespaces(self, element):
        return element.strip()

    def preprocesssing(self, element):
        #element = self.remove_punctuation(element)
        #element = self.number_to_text(element)
        #element = self.lower_case(element)    
        #element = self.traductor(element)
        #assert type(element) == type('')
        #element = self.remove_whitespaces(element)
        return element
    
    def traduct(self):
        for head in self.header_tr:
            self.X['url_{}'.format(head)] = self.X[head].apply(self.traductor)
        return self.X
    
    def operation(self):
        self.X = self.X[self.header+self.header_tr]
        self.X['cat_struc'] = self.X.apply(lambda x: self.categorie_structure(x['cat_struc'], x['cat_autre_struc']), axis=1)
        for head in self.header_tr:
            self.X['{}_trad'.format(head)] = self.X[head].apply(self.preprocesssing)
        #self.X.to_csv(RefEnv().src.format('Transformed/prep_df1_clean_trad.csv'))
        return self.X

In [9]:
transform = Transformation()
X = transform.traduct()
to_keep = ['key_main','url_prez_struc', 'url_prez_produit_struc', 'url_prez_marche_struc', 'url_prez_zone_struc', 'url_prez_objectif_struc', 'url_prez_innovante_struc', 'url_prez_duplicable_struc', 'url_prez_durable_struc']
X = X[to_keep]
X_head = X.head()

/Users/py3/Google Drive (mohamed.diabakhate@digital-africa.co)/PROJET DATA/DATA/Transformed/traduct.json


In [ ]:
#index = index + 1
target = X[~X['key_main'].isin(keys)]
Y = target.to_dict(orient = 'records')
nbr_records = len(Y)
Y[1].keys()
print(nbr_records)
output = RefEnv().transformed.format('traduct_{}.json'.format(index))
print(output)

def get_translate(element):
    resp = requests.get(element)
    traduction = resp.json()['translations'][0]['text']
    return traduction

def get_translate_(element):
    return element

t0= time()
urls = ['url_prez_struc', 'url_prez_produit_struc', 'url_prez_marche_struc', 'url_prez_zone_struc', 'url_prez_objectif_struc', 'url_prez_innovante_struc', 'url_prez_duplicable_struc', 'url_prez_durable_struc']
records = []
for i, element in enumerate(Y):
    record = {'key_main':element['key_main']}
    record.update({url: get_translate(element[url]) for url in urls})
    records.append(record)
    with open(output, 'w') as outfile:
        json.dump(records, outfile)
    print('elapsed time {}, remaining {}'.format(round((time()-t0)/60), nbr_records-(i+1))) 
print(output)

119
/Users/py3/Google Drive (mohamed.diabakhate@digital-africa.co)/PROJET DATA/DATA/Transformed/traduct_3.json
elapsed time 0, remaining 118
elapsed time 0, remaining 117
elapsed time 0, remaining 116
elapsed time 1, remaining 115
elapsed time 1, remaining 114
elapsed time 1, remaining 113
elapsed time 1, remaining 112
elapsed time 1, remaining 111
elapsed time 1, remaining 110
elapsed time 1, remaining 109
elapsed time 2, remaining 108
elapsed time 2, remaining 107
elapsed time 2, remaining 106
elapsed time 2, remaining 105
elapsed time 3, remaining 104
elapsed time 3, remaining 103
elapsed time 3, remaining 102
elapsed time 3, remaining 101
elapsed time 3, remaining 100
elapsed time 3, remaining 99
elapsed time 3, remaining 98
elapsed time 3, remaining 97
elapsed time 3, remaining 96
elapsed time 3, remaining 95
elapsed time 4, remaining 94
elapsed time 4, remaining 93
elapsed time 4, remaining 92
elapsed time 4, remaining 91
elapsed time 4, remaining 90
elapsed time 4, remaining 89


In [42]:
import pandas

output_0 = RefEnv().transformed.format('traduct_first.json')
output_1 = RefEnv().transformed.format('traduct_1.json')
output_2 = RefEnv().transformed.format('traduct_2.json')
output_3 = RefEnv().transformed.format('traduct_3.json')

key_converted_0 = set(pandas.read_json(output_0)['key_main'])
key_converted_1 = set(pandas.read_json(output_1)['key_main'])
key_converted_2 = set(pandas.read_json(output_2)['key_main'])
key_converted_3 = set(pandas.read_json(output_3)['key_main'])


keys= key_converted_0.union(key_converted_1).union(key_converted_2).union(key_converted_3)

print(len(keys))
to_fix = set()
to_fix.add('http://skedit.info/_lifeta_gaston')
keys = keys.union(to_fix)
print(to_fix)
print(len(keys))

In [64]:
pandas.DataFrame(Y)['key_main'][0]

'http://skedit.info/_lifeta_gaston'

In [72]:
df = pandas.read_json(output_0).append(pandas.read_json(output_1)).append(pandas.read_json(output_2)).append(pandas.read_json(output_3))
df = df.to_dict(orient = 'records')
output_ = '/Volumes/GoogleDrive/My Drive/PROJET DATA/DATA/Transformed/traduct.json'
with open(output_, 'w') as outfile:
    json.dump(df, outfile)